In [1]:
import pandas as pd
import numpy as np

In [16]:
df = pd.read_csv("../BP_data_table2_new.csv", index_col=0)

/home/ephlius/.virtualenvs/3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ephlius/.virtualenvs/3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [36]:
df.head()

,patientunitstayid,chartoffset,entryoffset,heartrate,respiratoryrate,spo2,nibp_systolic,nibp_diastolic,nibp_mean,temperature,temperaturelocation,ibp_systolic,ibp_diastolic,ibp_mean
0,141168,6,6,140.0,NaN,NaN,82.0,59.0,NaN,NaN,NaN,NaN,NaN,67.0
1,141168,21,21,70.0,NaN,94.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.0
2,141168,36,36,70.0,NaN,94.0,105.0,59.0,NaN,NaN,NaN,NaN,NaN,75.0
3,141168,51,51,140.0,NaN,94.0,103.0,65.0,NaN,NaN,NaN,NaN,NaN,80.0
4,141168,66,66,140.0,NaN,92.0,99.0,68.0,NaN,NaN,NaN,NaN,NaN,80.0


In [29]:
#df[(df["patientunitstayid"]==141194)&(df['chartoffset']>=0)&(df['chartoffset']<=1440)]

In [37]:
df_group = df.groupby("patientunitstayid")

In [33]:
def calc_percent(df, timerange=[0, 1440], col='ibp_systolic', valuerange=[115, 125]):
    start = timerange[0]
    end = timerange[1]
    temp = df[(df['chartoffset']>=start) & (df['chartoffset']<=end)]
    non_na = temp[temp[col].notnull()]
    num_nonna = non_na.shape[0]
    if num_nonna==0:
        return np.nan
    value_min, value_max = valuerange[0], valuerange[1]
    range_num = non_na[(non_na[col]>=value_min) & (non_na[col]<=value_max)].shape[0]
    return range_num / num_nonna

In [38]:
def calc_percent_below(df, timerange=[0, 1440], col='ibp_systolic', valuerange=[70, 80]):
    start = timerange[0]
    end = timerange[1]
    temp = df[(df['chartoffset']>=start) & (df['chartoffset']<=end)]
    non_na = temp[temp[col].notnull()]
    num_nonna = non_na.shape[0]
    if num_nonna==0:
        return np.nan
    value_min= valuerange[0]
    below_num = non_na[(non_na[col]<value_min)].shape[0]
    return below_num/num_nonna

In [39]:
def calc_percent_above(df, timerange=[0, 1440], col='ibp_systolic', valuerange=[70, 80]):
    start = timerange[0]
    end = timerange[1]
    temp = df[(df['chartoffset']>=start) & (df['chartoffset']<=end)]
    non_na = temp[temp[col].notnull()]
    num_nonna = non_na.shape[0]
    if num_nonna==0:
        return np.nan
    value_max= valuerange[1]
    above_num = non_na[(non_na[col]>value_max)].shape[0]
    return above_num/num_nonna

In [40]:
below = df_group.apply(lambda x: calc_percent_below(x, timerange=[0, 1440], col='ibp_mean', valuerange=[70, 80]))
above = df_group.apply(lambda x: calc_percent_above(x, timerange=[0, 1440], col='ibp_mean', valuerange=[70, 80]))

In [41]:
below = pd.DataFrame(below).reset_index()
above = pd.DataFrame(above).reset_index()
below.columns = ["patientunitstayid", "propBelow_map_24"]
above.columns = ["patientunitstayid", "propAbove_map_24"]
result_df = below.copy()
result_df['propAbove_map_24'] = above['propAbove_map_24']
result_df['prop_map_24'] = 1-result_df['propAbove_map_24'] -result_df['propBelow_map_24']

In [34]:
result_df.head(2)

,patientunitstayid,propBelow_map_24,propAbove_map_24,prop_map_24,propBelow_asp_24,propAbove_asp_24,prop_asp_24,propBelow_adp_24,propAbove_adp_24,prop_adp_24
0,141168,0.1,0.5,0.4,NaN,NaN,NaN,NaN,NaN,NaN
1,141194,1.0,0.0,0.0,0.875,0.0,0.125,1.0,0.0,0.0


In [42]:
sbp_24_below = df_group.apply(lambda x: calc_percent_below(x, timerange=[0, 1440], col='ibp_systolic', valuerange=[110,120 ]))
sbp_24_below = pd.DataFrame(sbp_24_below).reset_index()
sbp_24_below.columns = ["patientunitstayid", "propBelow_asp_24"]
sbp_24_above = df_group.apply(lambda x: calc_percent_above(x, timerange=[0, 1440], col='ibp_systolic', valuerange=[110,120 ]))
sbp_24_above = pd.DataFrame(sbp_24_above).reset_index()
sbp_24_above.columns = ["patientunitstayid", "propAbove_asp_24"]
result_df['propBelow_asp_24'] = sbp_24_below['propBelow_asp_24']
result_df['propAbove_asp_24'] = sbp_24_above['propAbove_asp_24']
result_df['prop_asp_24'] = 1-result_df['propAbove_asp_24'] -result_df['propBelow_asp_24']

In [43]:
adp_24_below = df_group.apply(lambda x: calc_percent_below(x, timerange=[0, 1440], col='ibp_diastolic', valuerange=[50,60]))
adp_24_below = pd.DataFrame(adp_24_below).reset_index()
adp_24_below.columns = ["patientunitstayid", "propBelow_adp_24"]
adp_24_above = df_group.apply(lambda x: calc_percent_above(x, timerange=[0, 1440], col='ibp_diastolic', valuerange=[50,60]))
adp_24_above = pd.DataFrame(adp_24_above).reset_index()
adp_24_above.columns = ["patientunitstayid", "propAbove_adp_24"]
result_df['propBelow_adp_24'] = adp_24_below['propBelow_adp_24']
result_df['propAbove_adp_24'] = adp_24_above['propAbove_adp_24']
result_df['prop_adp_24'] = 1-result_df['propAbove_adp_24'] -result_df['propBelow_adp_24']

In [45]:
result_df.head()

,patientunitstayid,propBelow_map_24,propAbove_map_24,prop_map_24,propBelow_asp_24,propAbove_asp_24,prop_asp_24,propBelow_adp_24,propAbove_adp_24,prop_adp_24
0,141168,0.100000,0.300000,0.600000,NaN,NaN,NaN,NaN,NaN,NaN
1,141194,0.958333,0.000000,0.041667,0.750000,0.041667,0.208333,1.000000,0.000000,0.000000
2,141203,0.362637,0.318681,0.318681,NaN,NaN,NaN,NaN,NaN,NaN
3,141229,0.000000,0.750000,0.250000,NaN,NaN,NaN,NaN,NaN,NaN
4,141233,0.345794,0.149533,0.504673,0.327103,0.457944,0.214953,0.906542,0.009346,0.084112


In [46]:
result_df.to_csv("./csv_folder/eicu_prop_24h.csv")

# 48hour

In [47]:
below_48 = df_group.apply(lambda x: calc_percent_below(x, timerange=[0, 2880], col='ibp_mean', valuerange=[70, 80]))
above_48 = df_group.apply(lambda x: calc_percent_above(x, timerange=[0, 2880], col='ibp_mean', valuerange=[70, 80]))
below_48 = pd.DataFrame(below_48).reset_index()
above_48 = pd.DataFrame(above_48).reset_index()
below_48.columns = ["patientunitstayid", "propBelow_map_48"]
above_48.columns = ["patientunitstayid", "propAbove_map_48"]
result_df_48 = below_48.copy()
result_df_48['propAbove_map_48'] = above_48['propAbove_map_48']
result_df_48['prop_map_48'] = 1-result_df_48['propAbove_map_48'] -result_df_48['propBelow_map_48']

In [48]:
sbp_48_below = df_group.apply(lambda x: calc_percent_below(x, timerange=[0, 2880], col='ibp_systolic', valuerange=[110,120 ]))
sbp_48_below = pd.DataFrame(sbp_48_below).reset_index()
sbp_48_below.columns = ["patientunitstayid", "propBelow_asp_48"]
sbp_48_above = df_group.apply(lambda x: calc_percent_above(x, timerange=[0, 2880], col='ibp_systolic', valuerange=[110,120 ]))
sbp_48_above = pd.DataFrame(sbp_48_above).reset_index()
sbp_48_above.columns = ["patientunitstayid", "propAbove_asp_48"]
result_df_48['propBelow_asp_48'] = sbp_48_below['propBelow_asp_48']
result_df_48['propAbove_asp_48'] = sbp_48_above['propAbove_asp_48']
result_df_48['prop_asp_48'] = 1-result_df_48['propAbove_asp_48'] -result_df_48['propBelow_asp_48']

In [49]:
adp_48_below = df_group.apply(lambda x: calc_percent_below(x, timerange=[0, 2880], col='ibp_diastolic', valuerange=[50,60]))
adp_48_below = pd.DataFrame(adp_48_below).reset_index()
adp_48_below.columns = ["patientunitstayid", "propBelow_adp_48"]
adp_48_above = df_group.apply(lambda x: calc_percent_above(x, timerange=[0, 2880], col='ibp_diastolic', valuerange=[50,60]))
adp_48_above = pd.DataFrame(adp_48_above).reset_index()
adp_48_above.columns = ["patientunitstayid", "propAbove_adp_48"]
result_df_48['propBelow_adp_48'] = adp_48_below['propBelow_adp_48']
result_df_48['propAbove_adp_48'] = adp_48_above['propAbove_adp_48']
result_df_48['prop_adp_48'] = 1-result_df_48['propAbove_adp_48'] -result_df_48['propBelow_adp_48']

In [50]:
result_df_48.head()

,patientunitstayid,propBelow_map_48,propAbove_map_48,prop_map_48,propBelow_asp_48,propAbove_asp_48,prop_asp_48,propBelow_adp_48,propAbove_adp_48,prop_adp_48
0,141168,0.133333,0.266667,0.600000,0.600000,0.200000,0.200000,0.400000,0.200000,0.400000
1,141194,0.750000,0.153846,0.096154,0.745098,0.117647,0.137255,0.686275,0.156863,0.156863
2,141203,0.376238,0.316832,0.306931,NaN,NaN,NaN,NaN,NaN,NaN
3,141229,0.000000,0.833333,0.166667,NaN,NaN,NaN,NaN,NaN,NaN
4,141233,0.621359,0.077670,0.300971,0.650485,0.237864,0.111650,0.733010,0.009709,0.257282


In [51]:
result_df_48.to_csv("./csv_folder/eicu_prop_48h.csv")

# 72hours

In [52]:
below_72 = df_group.apply(lambda x: calc_percent_below(x, timerange=[0, 4320], col='ibp_mean', valuerange=[70, 80]))
above_72 = df_group.apply(lambda x: calc_percent_above(x, timerange=[0, 4320], col='ibp_mean', valuerange=[70, 80]))
below_72 = pd.DataFrame(below_72).reset_index()
above_72 = pd.DataFrame(above_72).reset_index()
below_72.columns = ["patientunitstayid", "propBelow_map_72"]
above_72.columns = ["patientunitstayid", "propAbove_map_72"]
result_df_72 = below_72.copy()
result_df_72['propAbove_map_72'] = above_72['propAbove_map_72']
result_df_72['prop_map_72'] = 1-result_df_72['propAbove_map_72'] -result_df_72['propBelow_map_72']

In [53]:
sbp_72_below = df_group.apply(lambda x: calc_percent_below(x, timerange=[0, 4320], col='ibp_systolic', valuerange=[110,120]))
sbp_72_below = pd.DataFrame(sbp_72_below).reset_index()
sbp_72_below.columns = ["patientunitstayid", "propBelow_asp_72"]
sbp_72_above = df_group.apply(lambda x: calc_percent_above(x, timerange=[0, 4320], col='ibp_systolic', valuerange=[110,120]))
sbp_72_above = pd.DataFrame(sbp_72_above).reset_index()
sbp_72_above.columns = ["patientunitstayid", "propAbove_asp_72"]
result_df_72['propBelow_asp_72'] = sbp_72_below['propBelow_asp_72']
result_df_72['propAbove_asp_72'] = sbp_72_above['propAbove_asp_72']
result_df_72['prop_asp_72'] = 1-result_df_72['propAbove_asp_72'] -result_df_72['propBelow_asp_72']

In [54]:
adp_72_below = df_group.apply(lambda x: calc_percent_below(x, timerange=[0, 4320], col='ibp_diastolic', valuerange=[50,60]))
adp_72_below = pd.DataFrame(adp_72_below).reset_index()
adp_72_below.columns = ["patientitstayid", "propBelow_adp_72"]
adp_72_above = df_group.apply(lambda x: calc_percent_above(x, timerange=[0, 4320], col='ibp_diastolic', valuerange=[50,60]))
adp_72_above = pd.DataFrame(adp_72_above).reset_index()
adp_72_above.columns = ["patientunitstayid", "propAbove_adp_72"]
result_df_72['propBelow_adp_72'] = adp_72_below['propBelow_adp_72']
result_df_72['propAbove_adp_72'] = adp_72_above['propAbove_adp_72']
result_df_72['prop_adp_72'] = 1-result_df_72['propAbove_adp_72'] -result_df_72['propBelow_adp_72']

In [56]:
result_df_72.head()

,patientunitstayid,propBelow_map_72,propAbove_map_72,prop_map_72,propBelow_asp_72,propAbove_asp_72,prop_asp_72,propBelow_adp_72,propAbove_adp_72,prop_adp_72
0,141168,0.133333,0.266667,0.600000,0.600000,0.200000,0.200000,0.400000,0.200000,0.400000
1,141194,0.750000,0.153846,0.096154,0.745098,0.117647,0.137255,0.686275,0.156863,0.156863
2,141203,0.376238,0.316832,0.306931,NaN,NaN,NaN,NaN,NaN,NaN
3,141229,0.000000,0.833333,0.166667,NaN,NaN,NaN,NaN,NaN,NaN
4,141233,0.555085,0.072034,0.372881,0.614407,0.207627,0.177966,0.639831,0.084746,0.275424


In [57]:
result_df_72.to_csv("./csv_folder/eicu_prop_72h.csv")

# below not used

In [68]:
mbp_48_inrange = df_group.apply(lambda x: calc_percent(x, timerange=[0, 2880], col='ibp_mean', valuerange=[78.27889, 98.53015]))
mbp_48_inrange = pd.DataFrame(mbp_48_inrange).reset_index()
mbp_48_inrange.columns = ["patientunitstayid", "mbp_48_inrange"]
result_df['mbp_48_inrange'] = mbp_48_inrange['mbp_48_inrange']

sbp_48_inrange = df_group.apply(lambda x: calc_percent(x, timerange=[0, 2880], col='ibp_systolic', valuerange=[122.4724, 154.3116]))
sbp_48_inrange = pd.DataFrame(sbp_48_inrange).reset_index()
sbp_48_inrange.columns = ["patientunitstayid", "sbp_48_inrange"]
result_df['sbp_48_inrange'] = sbp_48_inrange['sbp_48_inrange']

dbp_48_inrange = df_group.apply(lambda x: calc_percent(x, timerange=[0, 2880], col='ibp_diastolic', valuerange=[60.39196, 79.55276]))
dbp_48_inrange = pd.DataFrame(dbp_48_inrange).reset_index()
dbp_48_inrange.columns = ["patientunitstayid", "dbp_48_inrange"]
result_df['dbp_48_inrange'] = dbp_48_inrange['dbp_48_inrange']

mbp_72_inrange = df_group.apply(lambda x: calc_percent(x, timerange=[0, 4320], col='ibp_mean', valuerange=[79.39698, 93.25628]))
mbp_72_inrange = pd.DataFrame(mbp_72_inrange).reset_index()
mbp_72_inrange.columns = ["patientunitstayid", "mbp_72_inrange"]
result_df['mbp_72_inrange'] = mbp_72_inrange['mbp_72_inrange']

sbp_72_inrange = df_group.apply(lambda x: calc_percent(x, timerange=[0, 4320], col='ibp_systolic', valuerange=[123.4673, 153.3166]))
sbp_72_inrange = pd.DataFrame(sbp_72_inrange).reset_index()
sbp_72_inrange.columns = ["patientunitstayid", "sbp_72_inrange"]
result_df['sbp_72_inrange'] = sbp_72_inrange['sbp_72_inrange']

dbp_72_inrange = df_group.apply(lambda x: calc_percent(x, timerange=[0, 4320], col='ibp_diastolic', valuerange=[61.01005, 82.64322]))
dbp_72_inrange = pd.DataFrame(dbp_72_inrange).reset_index()
dbp_72_inrange.columns = ["patientunitstayid", "dbp_72_inrange"]
result_df['dbp_72_inrange'] = dbp_72_inrange['dbp_72_inrange']

In [69]:
result_df.head()

,patientunitstayid,mbp_24_inrange,sbp_24_inrange,dbp_24_inrange,mbp_48_inrange,sbp_48_inrange,dbp_48_inrange,mbp_72_inrange,sbp_72_inrange,dbp_72_inrange
0,141168,0.700000,NaN,NaN,0.533333,0.200000,0.200000,0.466667,0.200000,0.200000
1,141194,0.000000,0.041667,0.000000,0.153846,0.098039,0.078431,0.153846,0.098039,0.078431
2,141203,0.417582,NaN,NaN,0.316832,NaN,NaN,0.247525,NaN,NaN
3,141229,1.000000,NaN,NaN,0.500000,NaN,NaN,0.500000,NaN,NaN
4,141233,0.364486,0.457944,0.037383,0.131068,0.150485,0.009709,0.122881,0.114407,0.084746


In [73]:
result_df[result_df['mbp_24_inrange'].notnull()].shape

(20884, 10)

In [74]:
df1 = pd.read_csv("./csv_folder/eicu_table1.csv", index_col=0)

In [75]:
df1.head()

,X,patientunitstayid,age,gender,ethnicity,unittype,admissionheight,admissionweight,icu_mort,icu_los,...,exclude_temperature,ibp_systolic_24h,ibp_systolic_48h,ibp_systolic_72h,ibp_diastolic_24h,ibp_diastolic_48h,ibp_diastolic_72h,ibp_mean_24h,ibp_mean_48h,ibp_mean_72h
1,0,141168,58,Female,Caucasian,Med-Surg ICU,152.4,84.3,EXPIRED,2.4972,...,False,NaN,98.0,98.0,NaN,56.0,56.0,79.5,79.0,79.0
2,1,141194,56,Male,Caucasian,CTICU,180.3,73.9,ALIVE,3.3423,...,False,99.0,98.0,98.0,40.0,42.0,42.0,56.0,59.0,59.0
3,2,141203,65,Female,Caucasian,Med-Surg ICU,160.0,70.2,ALIVE,1.2979,...,False,NaN,NaN,NaN,NaN,NaN,NaN,73.0,73.0,73.0
5,4,141233,69,Female,Caucasian,CTICU,165.1,61.7,ALIVE,10.8923,...,False,118.0,98.0,99.0,42.0,44.0,46.0,74.0,64.0,66.0
6,5,141244,47,Male,Caucasian,CTICU,180.3,92.3,ALIVE,2.6631,...,False,136.0,136.0,136.0,60.0,60.0,60.0,86.0,86.0,86.0


In [76]:
df3 = pd.merge(df1, result_df, how='left', on=['patientunitstayid'])

In [77]:
df3.head()

,X,patientunitstayid,age,gender,ethnicity,unittype,admissionheight,admissionweight,icu_mort,icu_los,...,ibp_mean_72h,mbp_24_inrange,sbp_24_inrange,dbp_24_inrange,mbp_48_inrange,sbp_48_inrange,dbp_48_inrange,mbp_72_inrange,sbp_72_inrange,dbp_72_inrange
0,0,141168,58,Female,Caucasian,Med-Surg ICU,152.4,84.3,EXPIRED,2.4972,...,79.0,0.700000,NaN,NaN,0.533333,0.200000,0.200000,0.466667,0.200000,0.200000
1,1,141194,56,Male,Caucasian,CTICU,180.3,73.9,ALIVE,3.3423,...,59.0,0.000000,0.041667,0.000000,0.153846,0.098039,0.078431,0.153846,0.098039,0.078431
2,2,141203,65,Female,Caucasian,Med-Surg ICU,160.0,70.2,ALIVE,1.2979,...,73.0,0.417582,NaN,NaN,0.316832,NaN,NaN,0.247525,NaN,NaN
3,4,141233,69,Female,Caucasian,CTICU,165.1,61.7,ALIVE,10.8923,...,66.0,0.364486,0.457944,0.037383,0.131068,0.150485,0.009709,0.122881,0.114407,0.084746
4,5,141244,47,Male,Caucasian,CTICU,180.3,92.3,ALIVE,2.6631,...,86.0,0.684211,0.684211,0.789474,0.631579,0.631579,0.421053,0.578947,0.631579,0.421053


In [90]:
# df3.plot(x='mbp_24_inrange', y='hosp_mort')
df3['hospital_mortality']= 1
df3.loc[df3['hosp_mort']=='ALIVE', 'hospital_mortality']=0

In [91]:
df3[['hosp_mort','hospital_mortality']].head(2)

,hosp_mort,hospital_mortality
0,EXPIRED,1
1,ALIVE,0


In [95]:
df3.to_csv("./csv_folder/eicu_table1_wth_range.csv")